In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("LIUM/tedlium", "release1", split = "train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.24k [00:00<?, ?B/s]

tedlium.py:   0%|          | 0.00/17.6k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

The repository for LIUM/tedlium contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/LIUM/tedlium.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

test.tar.gz:   0%|          | 0.00/307M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
transcriptions = dataset['text']

In [7]:
import csv

with open("output.csv", "w", newline = "") as file:
    writer = csv.writer(file)
    for transcription in transcriptions:
        writer.writerow([transcription])

NameError: name 'transcriptions' is not defined

In [ ]:
import re

def clean_text(text):
    # Remove content within curly braces, angle brackets, and parentheses
    cleaned_text = re.sub(r'\{[^}]*\}|\<[^>]*\>|\([^)]*\)', '', text)

    # Remove any non-English characters
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', cleaned_text)

    # Remove extra spaces and strip leading/trailing whitespace
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

In [ ]:
data = []
with open("output.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

modified_data = []

for i in range(len(data)):
    modified_data.append(clean_text(data[i][0].lower()))

with open("normalized_output.csv", "w", newline = "") as file:
    writer = csv.writer(file)
    for transcription in modified_data:
        writer.writerow([transcription])

In [74]:
# start here if have normalized_output.csv already
import csv

data = []
with open("normalized_output.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

In [75]:
print(data[:2])

[['and whose purpose in following the path of the prophet is to make ourselves as much like the'], ['and the prophet in one of his sayings said adorn yourselves with the attributes of god and because god himself said that the primary attribute of his is compassion']]


In [76]:
print(len(data))

56803


In [77]:
!pip install textstat

In [78]:
import numpy as np
import re
import textstat
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import spacy

In [104]:
# Flatten the 2D data into a 1D list of strings
flattened_data = [item for sublist in data for item in sublist]

# Load English NLP model
nlp = spacy.load("en_core_web_sm")

# Function to calculate lexical diversity
def lexical_diversity(text):
    words = text.split()
    return len(set(words)) / len(words) if words else 0

# Function to detect filler words
def count_filler_words(text):
    filler_words = {
        # Common hesitation sounds
        "uh", "uhh", "uhhh", "uhhhh",
        "um", "umm", "ummm", "ummmm",
        "ah", "ahh", "ahhh", "ahhhh",
        "eh", "ehh", "ehhh", "ehhhh",
        "erm", "er", "hmm", "hmmm", "hmmmm",
        "huh", "huhh", "huhhh",
        "mmm", "mm", "mhmm", "mhm", "mmhmm", "mmhm",

        # Casual interjections
        "like", "literally", "basically", "actually", "seriously", "honestly", "clearly",
        "obviously", "totally", "really", "definitely", "frankly", "legit", "essentially",
        "technically", "apparently", "obvi", "prob", "probably", "possibly", "maybe", "I guess",
        "I suppose", "I don't know", "sort of", "kind of", "kinda", "gonna", "wanna", "lemme",
        "innit", "right", "you see", "know what I mean", "and stuff", "and things", "whatnot",
        "etcetera", "et cetera", "etc", "somehow", "someway", "whatever", "whichever", "who knows",
        "or whatever", "or anything", "or whatnot", "just saying", "I mean to say", "I guess", "yeah",
        "well", "ok", "okay", "alright", "anyway", "anyways", "actually", "basically", "stuff like that",
        "things like that", "or something", "and", "but", "so", "just", "just saying", "I mean",
        "but like", "like I said", "you know what I’m saying", "or whatever", "sort of", "kind of",
        "you know", "right?", "okay so", "and you know", "but you know", "uh, yeah", "yeah, uh",
        "like I mentioned", "to be honest", "you know what I mean",

        # Overuse of disclaimers and qualifiers
        "maybe", "perhaps", "probably", "possibly", "I feel like", "I believe", "I think",
        "I’d say", "I suppose", "it seems like", "it looks like", "I’m not sure but", "I could be wrong but",
        "I don’t know", "to be honest", "to be fair", "to be frank", "honestly", "just so you know",
        "you know what I mean", "I don’t mean to sound rude but", "I’m just saying", "for the most part",
        "more or less", "in a way", "kind of", "I think", "you get me", "if I’m being honest",
        "for what it's worth", "I believe", "I suppose", "I could be wrong", "if I’m being real", "to put it simply",
        "basically", "in a sense", "from my perspective", "long story short", "to be honest with you",
        "to tell you the truth", "I'm not sure if you’ll agree with me but",

        # Discourse markers (weak sentence starters and fillers)
        "so", "well", "okay", "alright", "anyway", "anyways", "actually", "basically",
        "stuff like that", "things like that", "whatnot", "etcetera", "et cetera", "etc", "somehow",
        "someway", "whatever", "whichever", "who knows", "and stuff", "and things", "or whatever",

        # Phrases that dilute or weaken statements
        "at the end of the day", "if that makes sense", "for the most part", "more or less",
        "all things considered", "long story short", "I mean", "not to be rude but", "no offense but",
        "if I'm being honest", "to put it simply", "let's be real", "to tell you the truth", "you know what I mean",

        # Casual fillers and crutches
        "basically", "you know", "I mean", "like I said", "right", "you see", "I guess",
        "literally", "seriously", "just", "and so on", "and stuff", "and things", "or whatever",
        "and all", "and everything", "that sort of thing", "that kind of thing", "so yeah", "and I mean",
        "right so", "if you will", "to some extent", "you know what I’m saying", "kind of like", "pretty much",

        # Additional hesitation and softening words
        "huh", "uhhuh", "hmm", "mm", "ahh", "err", "well", "you know", "you see", "okay", "right?",
        "basically", "actually", "so", "uhoh", "umhm", "I don't know", "right", "umm", "you know what I mean"
    }
    words = re.findall(r'\b\w+\b', text.lower())
    return sum(1 for word in words if word in filler_words)

# Compute readability score
def readability_score(text):
    return textstat.flesch_reading_ease(text)

# Compute TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(flattened_data)

def compute_tfidf(text):
    tfidf_matrix = vectorizer.transform(text)
    return tfidf_matrix.toarray()

scaler = MinMaxScaler()

def compute_score(data):
    # Extract Features
    tfidf_scores = compute_tfidf(data)
    readability_scores = np.array([readability_score(text) for text in data])
    lexical_diversity_scores = np.array([lexical_diversity(text) for text in data])
    filler_word_counts = np.array([count_filler_words(text) for text in data])

    # Normalize features
    normalized_tfidf = scaler.fit_transform(tfidf_scores.mean(axis=1).reshape(-1, 1))
    normalized_readability = scaler.fit_transform(readability_scores.reshape(-1, 1))
    normalized_lexical_diversity = scaler.fit_transform(lexical_diversity_scores.reshape(-1, 1))
    normalized_filler_words = scaler.fit_transform(filler_word_counts.reshape(-1, 1))

    # Compute final score as weighted sum (adjust weights based on importance)
    final_scores = (0.50 * normalized_tfidf +
                    0.20 * normalized_readability +
                    0.20 * normalized_lexical_diversity -
                    0.50 * normalized_filler_words)
    return final_scores


# Classification function
def classify_score(score):
    if score <= 0.20:
        return "weak"
    elif score <= 0.40:
        return "average"
    else:
        return "strong"

# # Display results
# final_scores = compute_score(flattened_data)
# for i, argument in enumerate(flattened_data[:10]):
#     print(f"Argument: {argument}")
#     print(f"Score: {final_scores[i][0]:.4f}")
#     print(f"Classification: {classify_score(final_scores[i][0])}")
#     print()

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [105]:
import re

def clean_text(data):
    clean_text = []
    for text_list in data:
        # Assuming each item in 'text_list' is a string, take the first element (text[0])
        text = text_list[0]

        # Remove content within curly braces, angle brackets, and parentheses
        cleaned_text = re.sub(r'\{[^}]*\}|\<[^>]*\>|\([^)]*\)', '', text)

        # Remove any non-English characters (anything except a-zA-Z and spaces)
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', cleaned_text)

        # Remove extra spaces and strip leading/trailing whitespace
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

        clean_text.append(cleaned_text.lower())

    return clean_text

In [106]:
best_lines = [
    ["The evidence clearly shows that we must act now to prevent irreversible damage."],
    ["If we fail to address this issue, we risk leaving future generations to bear the consequences."],
    ["The argument in favor of action is not just moral, but practical."],
    ["I challenge my opponent to explain why their approach will not result in disastrous long-term effects."],
    ["Through careful analysis and extensive data, we've established a clear path forward."],
    ["We cannot afford to ignore the overwhelming scientific consensus on this matter."],
    ["This is not just a policy debate; it's a debate about our values and our future."],
    ["As you can see, my opponent's argument relies on faulty assumptions and ignores critical facts."],
    ["The solution to this problem is clear, and it lies in the hands of those willing to take bold action."],
    ["We have a moral obligation to protect the most vulnerable, and that is what this policy does."],
    ["In conclusion, we must prioritize the long-term benefits over short-term gains for a sustainable future."]
]

In [107]:
normalized_best = clean_text(best_lines)

In [108]:
normalized_best[:2]

['the evidence clearly shows that we must act now to prevent irreversible damage',
 'if we fail to address this issue we risk leaving future generations to bear the consequences']

In [109]:
final_scores = compute_score(normalized_best)
final_scores = np.clip(final_scores, 0, 1)
for i, argument in enumerate(normalized_best):
    print(f"Argument: {argument}")
    print(f"Score: {final_scores[i][0]:.4f}")
    print(f"Classification: {classify_score(final_scores[i][0])}")
    print()

Argument: the evidence clearly shows that we must act now to prevent irreversible damage
Score: 0.3620
Classification: average

Argument: if we fail to address this issue we risk leaving future generations to bear the consequences
Score: 0.6832
Classification: strong

Argument: the argument in favor of action is not just moral but practical
Score: 0.0347
Classification: weak

Argument: i challenge my opponent to explain why their approach will not result in disastrous longterm effects
Score: 0.6228
Classification: strong

Argument: through careful analysis and extensive data weve established a clear path forward
Score: 0.4987
Classification: strong

Argument: we cannot afford to ignore the overwhelming scientific consensus on this matter
Score: 0.4224
Classification: strong

Argument: this is not just a policy debate its a debate about our values and our future
Score: 0.0000
Classification: weak

Argument: as you can see my opponents argument relies on faulty assumptions and ignores cr

In [87]:
worst_lines = [
    ["Um, I think, like, we could maybe, you know, try something, but I'm not sure."],
    ["Well, I don't really have much to add, but, like, it's kind of obvious, right?"],
    ["Honestly, I mean, uh, maybe it's not the best solution, but it might work?"],
    ["Like, we could do this, or we could do that, but it's hard to say which one's better."],
    ["So, um, it's sort of important, I guess, to think about the problem, but I'm not really sure how to fix it."],
    ["Uh, I think we should, you know, consider a few things, but it's not a huge deal."],
    ["Well, I guess, like, it's not perfect, but I think it could be okay?"],
    ["I'm not really sure about all the details, but, like, you get the idea."],
    ["To be honest, I don’t know what the best solution is, but we should figure something out."],
    ["So yeah, uh, maybe we should just think about it and, like, see what happens."]
]